# Implementing binary decision trees
The goal of this notebook is to implement your own binary decision tree classifier. You will:
    
* Use Pandas to do some feature engineering.
* Transform categorical variables into binary variables.
* Write a function to compute the number of misclassified examples in an intermediate node.
* Write a function to find the best feature to split on.
* Build a binary decision tree from scratch.
* Make predictions using the decision tree.
* Evaluate the accuracy of the decision tree.
* Visualize the decision at the root node.

**Important Note**: In this assignment, we will focus on building decision trees where the data contain **only binary (0 or 1) features**. This allows us to avoid dealing with:
* Multiple intermediate nodes in a split
* The thresholding issues of real-valued features.

This assignment **may be challenging**, so brace yourself :)

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split

## Data Preprocessing

### Load LendingClub dataset
**1**. We will be using a dataset from the [LendingClub](https://www.lendingclub.com/). A parsed and cleaned form of the dataset is availiable [here](https://github.com/learnml/machine-learning-specialization-private).

In [3]:
DATA_DIR = os.path.join('data')

print(os.listdir(DATA_DIR))

['.DS_Store', 'lending-club-data.sframe', 'module-5-assignment-1-validation-idx.json', 'module-5-assignment-1-train-idx.json', 'module-5-assignment-2-test-idx.json', 'c3w3_Quiz.jpeg', 'lending-club-data.csv', 'module-5-assignment-2-train-idx.json']


In [4]:
loans = pd.read_csv(os.path.join(DATA_DIR, 'lending-club-data.csv'))

/Users/rocioliu/Library/Python/3.7/lib/python/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
loans.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1.0,1.0,1.0,0,8.14350,20141201T000000,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,0.8,1.0,1.0,1.0,1,2.39320,20161201T000000,1,1,1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1.0,1.0,1.0,0,8.25955,20141201T000000,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,0.2,1.0,1.0,1.0,0,8.27585,20141201T000000,0,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,0.8,1.0,1.0,1.0,0,5.21533,20141201T000000,1,1,1


**2**. Like the previous assignment, reassign the labels to have +1 for a safe loan, and -1 for a risky (bad) loan.

In [6]:
loans['bad_loans'].value_counts()

0    99457
1    23150
Name: bad_loans, dtype: int64

In [7]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x: +1 if x == 0 else -1)

loans.drop('bad_loans', axis=1, inplace=True)

In [8]:
loans['safe_loans'].value_counts()

 1    99457
-1    23150
Name: safe_loans, dtype: int64

**3**. Unlike the previous assignment, we will only be considering these four features:

In [9]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'

Extract these feature columns from the dataset, and discard the rest of the feature columns.

In [10]:
loans = loans[features + [target]]

In [11]:
loans.head()

,grade,term,home_ownership,emp_length,safe_loans
0,B,36 months,RENT,10+ years,1
1,C,60 months,RENT,< 1 year,-1
2,C,36 months,RENT,10+ years,1
3,C,36 months,RENT,10+ years,1
4,A,36 months,RENT,3 years,1


Check if there are null values in the columns.

In [12]:
loans.isnull().sum()

grade                0
term                 0
home_ownership       0
emp_length        4091
safe_loans           0
dtype: int64

Transfrom null values into 'NaN'.

In [13]:
for col in loans:
    loans[col] = loans[col].fillna('NaN')

In [14]:
loans.isnull().sum()

grade             0
term              0
home_ownership    0
emp_length        0
safe_loans        0
dtype: int64

### Sample data to balance classes

**4**. Just as we did in the previous assignment, we will undersample the larger class (safe loans) in order to balance out our dataset.

**Since we would like to make sure our final result matches the correct result. We should use the specific training dataset and validdation dataset here.**

Then follow the following steps:

* Load the JSON files into the lists train_idx and validation_idx.
* Apply one-hot encoding to loans. Your tool may have a function for one-hot encoding. Alternatively, see #7 for implementation hints.
* Perform train/test split using train_idx and validation_idx. In Pandas, for instance:

We can regard the step below as the process of our imbalance dataset:

( **We merge the rebalance step and train/test split step into one step here. We do this only because we want to pass the quiz.**)

In [15]:
import json

with open(os.path.join(DATA_DIR, 'module-5-assignment-2-train-idx.json')) as json_file:
    train_idx = json.load(json_file)
    
with open(os.path.join(DATA_DIR, 'module-5-assignment-2-test-idx.json')) as json_file:
    test_idx = json.load(json_file)

**Except for this assignment, we can do as follows** ( *but we should skip this section in this assignment* ): 

---
As we explored above, our data is disproportionally full of safe loans.  Let's create two datasets: one with just the safe loans (`safe_loans_raw`) and one with just the risky loans (`risky_loans_raw`).

In [16]:
safe_loans_raw = loans[loans['safe_loans'] == +1]
risky_loans_raw = loans[loans['safe_loans'] == -1]

print(f"Number of safe loans: {len(safe_loans_raw)}")
print(f"Number of risky loans: {len(risky_loans_raw)}")
print()
print(f"Percentage of safe loans  : {round(len(safe_loans_raw)/(len(safe_loans_raw)+len(risky_loans_raw)), 2)}") 
print(f"Percentage of risky loans : {round(len(risky_loans_raw)/(len(safe_loans_raw)+len(risky_loans_raw)), 2)}")

Number of safe loans: 99457
Number of risky loans: 23150

Percentage of safe loans  : 0.81
Percentage of risky loans : 0.19


One way to combat class imbalance is to undersample the larger class until the class distribution is approximately half and half. Here, we will undersample the larger class (safe loans) in order to balance out our dataset. This means we are throwing away many data points. We used `random_state=1` so everyone gets the same results.

In [17]:
# Since there are fewer risky loans than safe loans, find the ratio of the sizes
# and use that percentage to undersample the safe loans.
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))

risky_loans = risky_loans_raw
safe_loans = safe_loans_raw.sample(frac=percentage, random_state=1)

# Append the risky_loans with the downsampled version of safe_loans
loans_data = risky_loans.append(safe_loans)

Now, let's verify that the resulting percentage of safe and risky loans are each nearly 50%.

In [18]:
print(f"Percentage of safe loans                 : {len(safe_loans) / float(len(loans_data))}")
print(f"Percentage of risky loans                : {len(risky_loans) / float(len(loans_data))}")
print(f"Total number of loans in our new dataset : {len(loans_data)}")

Percentage of safe loans                 : 0.5
Percentage of risky loans                : 0.5
Total number of loans in our new dataset : 46300


**Note:** There are many approaches for dealing with imbalanced data, including some where we modify the learning algorithm. These approaches are beyond the scope of this course, but some of them are reviewed in this [paper](http://ieeexplore.ieee.org/xpl/login.jsp?tp=&arnumber=5128907&url=http%3A%2F%2Fieeexplore.ieee.org%2Fiel5%2F69%2F5173046%2F05128907.pdf%3Farnumber%3D5128907 ). For this assignment, we use the simplest possible approach, where we subsample the overly represented class to get a more balanced dataset. In general, and especially when the data is highly imbalanced, we recommend using more advanced methods.

---

### Transform categorical data into binary features -- one-hot encoding
In this assignment, we will implement **binary decision trees** (decision trees for binary features, a specific case of categorical variables taking on two values, e.g., true/false). Since all of our features are currently categorical features, we want to turn them into binary features.

For instance, the **home_ownership** feature represents the home ownership status of the loanee, which is either `own`, `mortgage` or `rent`. For example, if a data point has the feature 
```
   {'home_ownership': 'RENT'}
```
we want to turn this into three features: 
```
 { 
   'home_ownership = OWN'      : 0, 
   'home_ownership = MORTGAGE' : 0, 
   'home_ownership = RENT'     : 1
 }
```

**5**. This technique of turning categorical variables into binary variables is called one-hot encoding. Perform one-hot encoding on the four features described above. You should now have 25 binary features.

In [19]:
# def one_hot_encoding(df, col_name):
#     for cla in sorted(df[col_name].unique()):
#         df[f"{col_name}_{cla}"] = df[col_name].apply(lambda x: 1 if x == cla else 0)
        
#     return df

In [19]:
loans = pd.get_dummies(loans, prefix=features)

In [20]:
loans.head()

,safe_loans,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,term_ 36 months,term_ 60 months,...,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year,emp_length_NaN
0,1,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,-1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [23]:
#loans.columns = [col.replace(' ', '_') for col in loans.columns]

In [21]:
features = list(loans.columns)
features.remove('safe_loans')

print(f"Number of features (after binarizing categorical variables) = {len(features)}")

Number of features (after binarizing categorical variables) = 25


Let's explore what one of these columns looks like:

In [22]:
print(len(loans['grade_A']), '\n')
print(loans['grade_A'][:10])

122607 

0    0
1    0
2    0
3    0
4    1
5    0
6    0
7    0
8    0
9    0
Name: grade_A, dtype: uint8


This column is set to 1 if the loan grade is A and 0 otherwise.

### Train-test split

**6**. Split the data into a train test split with 80% of the data in the training set and 20% of the data in the test set. Call these datasets **train_data** and **test_data**, respectively. 

**Since we would like to make sure our final result matches the correct result. We should use the specific training dataset and validdation dataset here.**

In [23]:
train_data = loans.iloc[train_idx]
test_data = loans.iloc[test_idx]

In [24]:
print(f"Number of training_data: {len(train_data)}")
print(f"Number of test_data: {len(test_data)}")
print()
print(f"Percentage of train_data: {round((len(train_data)/(len(train_data)+len(test_data))), 2)}")
print(f"Percentage of test_data : {round((len(test_data)/(len(train_data)+len(test_data))), 2)}")

Number of training_data: 37224
Number of test_data: 9284

Percentage of train_data: 0.8
Percentage of test_data : 0.2


**Except for this assignment, we can do as follows** ( *but we should skip this section in this assignment* ): 

---

In [25]:
train_dataset, test_dataset = train_test_split(loans, test_size=0.2, random_state=1)

In [26]:
print(f"Percentage of train_data: {round((len(train_dataset)/(len(train_dataset)+len(test_dataset))), 2)}")
print(f"Percentage of test_data : {round((len(test_dataset)/(len(train_dataset)+len(test_dataset))), 2)}")

Percentage of train_data: 0.8
Percentage of test_data : 0.2


---

**Checkpoint:** Make sure the following answers match up.

In [27]:
print(f"Total number of grade_A loans : {train_data['grade_A'].sum() + test_data['grade_A'].sum()}")
print(f"Expexted answer               : 6422")

Total number of grade_A loans : 6422
Expexted answer               : 6422


## Decision tree implementation

In this section, we will implement binary decision trees from scratch. There are several steps involved in building a decision tree. For that reason, we have split the entire assignment into several sections.

### Function to count number of mistakes while predicting majority class

Recall from the lecture that prediction at an intermediate node works by predicting the **majority class** for all data points that belong to this node.

Now, we will write a function that calculates the number of **missclassified examples** when predicting the **majority class**. This will be used to help determine which feature is the best to split on at a given node of the tree.

**Note**: Keep in mind that in order to compute the number of mistakes for a majority classifier, we only need the label (y values) of the data points in the node. 

** Steps to follow **:
* **Step 1:** Calculate the number of safe loans and risky loans.
* **Step 2:** Since we are assuming majority class prediction, all the data points that are **not** in the majority class are considered **mistakes**.
* **Step 3:** Return the number of **mistakes**.


Now, let us write the function `intermediate_node_num_mistakes` which computes the number of misclassified examples of an intermediate node given the set of labels (y values) of the data points contained in the node. Fill in the places where you find `## YOUR CODE HERE`. There are **three** places in this function for you to fill in.

In [28]:
def intermediate_node_num_mistakes(labels_in_node):
    '''
    labels_in_node: an array of y values
    '''
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0    
    # Count the number of 1's (safe loans)
    ## YOUR CODE HERE
    num_safe = (labels_in_node == 1).sum()
    
    # Count the number of -1's (risky loans)
    ## YOUR CODE HERE
    num_risky = (labels_in_node == -1).sum()
    
    # Return the number of mistakes that the majority classifier makes.
    ## YOUR CODE HERE    
    num_mistakes = num_risky if num_safe > num_risky else num_safe
    
    return num_mistakes

**8**. Because there are several steps in this assignment, we have introduced some stopping points where you can check your code and make sure it is correct before proceeding. To test your `intermediate_node_num_mistakes` function, run the following code until you get a **Test passed!**, then you should proceed. Otherwise, you should spend some time figuring out where things went wrong.

In [29]:
# Test case 1
example_labels = np.array([-1, -1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print('Test passed!')
else:
    print('Test 1 failed... try again!')

# Test case 2
example_labels = np.array([-1, -1, 1, 1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print('Test passed!')
else:
    print('Test 3 failed... try again!')
    
# Test case 3
example_labels = np.array([-1, -1, -1, -1, -1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print('Test passed!')
else:
    print('Test 3 failed... try again!')

Test passed!
Test passed!
Test passed!


### Function to pick best feature to split on

The function `best_splitting_feature` takes 3 arguments: 
1. The data (DataFrame of data which includes all of the feature columns and label column)
2. The features to consider for splits (a list of strings of column names to consider for splits)
3. The name of the target/label column (string)

The function will loop through the list of possible features, and consider splitting on each of them. It will calculate the classification error of each split and return the feature that had the smallest classification error when split on.

Recall that the **classification error** is defined as follows:
$$
\mbox{classification error} = \frac{\mbox{# mistakes}}{\mbox{# total examples}}
$$

**9**. Follow these steps to implement `best_splitting_feature`: 
* **Step 1:** Loop over each feature in the feature list
* **Step 2:** Within the loop, split the data into two groups: one group where all of the data has feature value 0 or False (we will call this the **left** split), and one group where all of the data has feature value 1 or True (we will call this the **right** split). Make sure the **left** split corresponds with 0 and the **right** split corresponds with 1 to ensure your implementation fits with our implementation of the tree building process.
* **Step 3:** Calculate the number of misclassified examples in both groups of data and use the above formula to compute the **classification error**.
* **Step 4:** If the computed error is smaller than the best error found so far, store this **feature and its error**.

This may seem like a lot, but we have provided pseudocode in the comments in order to help you implement the function correctly.

**Note:** Remember that since we are only dealing with binary features, we do not have to consider thresholds for real-valued features. This makes the implementation of this function much easier.

Fill in the places where you find `## YOUR CODE HERE`. There are **five** places in this function for you to fill in.

In [30]:
def best_splitting_feature(data, features, target):
    
    target_values = data[target]
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        # The left split will have all data points where the feature value is 0
        left_split = data[data[feature] == 0]
        
        # The right split will have all data points where the feature value is 1
        ## YOUR CODE HERE
        right_split =  data[data[feature] == 1]
            
        # Calculate the number of misclassified examples in the left split.
        # Remember that we implemented a function for this! (It was called intermediate_node_num_mistakes)
        # YOUR CODE HERE
        left_mistakes = intermediate_node_num_mistakes(left_split[target])           

        # Calculate the number of misclassified examples in the right split.
        ## YOUR CODE HERE
        right_mistakes = intermediate_node_num_mistakes(right_split[target])
            
        # Compute the classification error of this split.
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        ## YOUR CODE HERE
        error = (left_mistakes + right_mistakes) / num_data_points

        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        ## YOUR CODE HERE
        if error < best_error:
            best_error = error
            best_feature = feature
        
        
    return best_feature # Return the best feature we found

To test your `best_splitting_feature` function, run the following code:

In [31]:
if best_splitting_feature(train_data, features, 'safe_loans') == 'term_ 36 months':
    print('Test passed!')
else:
    print('Test failed... try again!')

Test passed!


### Building the tree

With the above functions implemented correctly, we are now ready to build our decision tree. Each node in the decision tree is represented as a dictionary which contains the following keys and possible values:

    { 
       'is_leaf'            : True/False.
       'prediction'         : Prediction at the leaf node.
       'left'               : (dictionary corresponding to the left tree).
       'right'              : (dictionary corresponding to the right tree).
       'splitting_feature'  : The feature that this node splits on.
    }

**10**. First, we will write a function that creates a leaf node given a set of target values. Fill in the places where you find `## YOUR CODE HERE`. There are **three** places in this function for you to fill in.

In [32]:
def create_leaf(target_values):
    '''
    target_values: df[target]
    '''
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True }   ## YOUR CODE HERE
    
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    
    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] = 1        ## YOUR CODE HERE
    else:
        leaf['prediction'] = -1       ## YOUR CODE HERE
        
    # Return the leaf node        
    return leaf 

We have provided a function that learns the decision tree recursively and implements 3 stopping conditions:
1. **Stopping condition 1:** All data points in a node are from the same class.
2. **Stopping condition 2:** No more features to split on.
3. **Additional stopping condition:** In addition to the above two stopping conditions covered in lecture, in this assignment we will also consider a stopping condition based on the **max_depth** of the tree. By not letting the tree grow too deep, we will save computational effort in the learning process. 

**11**. Now, we will write down the skeleton of the learning algorithm. Fill in the places where you find `## YOUR CODE HERE`. There are **seven** places in this function for you to fill in.

In [33]:
def decision_tree_create(data, features, target, current_depth = 0, max_depth = 10):
    '''
    data: DataFrame (with X and y)
    features: a list of strings
    target: string
    '''
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    print("--------------------------------------------------------------------")
    print(f"Subtree, depth = {current_depth} ({len(target_values)} data points).")


    # Stopping condition 1
    # (Check if there are mistakes at current node.
    # Recall you wrote a function intermediate_node_num_mistakes to compute this.)
    if intermediate_node_num_mistakes(target_values) == 0:  ## YOUR CODE HERE
        print("Stopping condition 1 reached.")
        # If not mistakes at current node, make current node a leaf node
        return create_leaf(target_values)
    
    # Stopping condition 2 (check if there are remaining features to consider splitting on)
    if remaining_features == 0:   ## YOUR CODE HERE
        print("Stopping condition 2 reached.")
        # If there are no remaining features to consider, make current node a leaf node
        return create_leaf(target_values)    
    
    # Additional stopping condition (limit tree depth)
    if current_depth >= max_depth:  ## YOUR CODE HERE
        print("Reached maximum depth. Stopping for now.")
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values)

    # Find the best splitting feature (recall the function best_splitting_feature implemented above)
    ## YOUR CODE HERE
    splitting_feature = best_splitting_feature(data, features, target)
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]     ## YOUR CODE HERE
    remaining_features.remove(splitting_feature)
    print(f"Split on feature {splitting_feature}. ({len(left_split)}, {len(right_split)})")
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print("Creating leaf node.")
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print("Creating leaf node.")
        ## YOUR CODE HERE
        return create_leaf(right_split[target])

        
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, current_depth + 1, max_depth)        
    ## YOUR CODE HERE
    right_tree = decision_tree_create(right_split, remaining_features, target, current_depth + 1, max_depth)

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

Here is a recursive function to count the nodes in your tree:

In [34]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

Run the following test code to check your implementation. Make sure you get **'Test passed'** before proceeding.

In [35]:
small_data_decision_tree = decision_tree_create(train_data, features, 'safe_loans', max_depth = 3)
if count_nodes(small_data_decision_tree) == 13:
    print('Test passed!')
else:
    print('Test failed... try again!')
    print('Number of nodes found                :', count_nodes(small_data_decision_tree))
    print('Number of nodes that should be there : 13' )

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 3 (1048 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Split on feature emp_length_NaN. (96, 5)
--------------------------------------------------------------------
Subtree, depth = 3 (96 data points)

In [36]:
small_data_decision_tree['splitting_feature']

'term_ 36 months'

In [37]:
small_data_decision_tree['left']

{'is_leaf': False,
 'prediction': None,
 'splitting_feature': 'grade_A',
 'left': {'is_leaf': False,
  'prediction': None,
  'splitting_feature': 'grade_B',
  'left': {'splitting_feature': None,
   'left': None,
   'right': None,
   'is_leaf': True,
   'prediction': -1},
  'right': {'splitting_feature': None,
   'left': None,
   'right': None,
   'is_leaf': True,
   'prediction': -1}},
 'right': {'is_leaf': False,
  'prediction': None,
  'splitting_feature': 'emp_length_NaN',
  'left': {'splitting_feature': None,
   'left': None,
   'right': None,
   'is_leaf': True,
   'prediction': 1},
  'right': {'splitting_feature': None,
   'left': None,
   'right': None,
   'is_leaf': True,
   'prediction': -1}}}

### Build the tree!

**12**. Now that all the tests are passing, we will train a tree model on the **train_data**. Limit the depth to 6 (**max_depth = 6**) to make sure the algorithm doesn't run for too long. Call this tree **my_decision_tree**. 

**Warning**: This code block may take 1-2 minutes to learn. 

In [38]:
# Make sure to cap the depth at 6 by using max_depth = 6
my_decision_tree = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
R

### Making predictions with a decision tree

**13**. As discussed in the lecture, we can make predictions from the decision tree with a simple recursive function. Below, we call this function `classify`, which takes in a learned `tree` and a test point `x` to classify.  We include an option `annotate` that describes the prediction path when set to `True`.

Fill in the places where you find `## YOUR CODE HERE`. There is **one** place in this function for you to fill in.

In [39]:
def classify(tree, x, annotate = False):
    '''
    tree: the tree model we built
    x: a couple of rows of DataFrame
    '''
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate: 
            print(f"At leaf, predicting {tree['prediction']}")
        return tree['prediction'] 
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate: 
            print(f"Split on {tree['splitting_feature']} = {split_feature_value}")
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            ### YOUR CODE HERE
            return classify(tree['right'], x, annotate)

Now, let's consider the first example of the test set and see what `my_decision_tree` model predicts for this data point.

In [40]:
test_data.iloc[0]

safe_loans                -1
grade_A                    0
grade_B                    0
grade_C                    0
grade_D                    1
grade_E                    0
grade_F                    0
grade_G                    0
term_ 36 months            0
term_ 60 months            1
home_ownership_MORTGAGE    0
home_ownership_OTHER       0
home_ownership_OWN         0
home_ownership_RENT        1
emp_length_1 year          0
emp_length_10+ years       0
emp_length_2 years         1
emp_length_3 years         0
emp_length_4 years         0
emp_length_5 years         0
emp_length_6 years         0
emp_length_7 years         0
emp_length_8 years         0
emp_length_9 years         0
emp_length_< 1 year        0
emp_length_NaN             0
Name: 24, dtype: int64

In [41]:
print(f'Predicted class: {classify(my_decision_tree, test_data.iloc[0])}')

Predicted class: -1


Let's add some annotations to our prediction to see what the prediction path was that lead to this predicted class:

In [42]:
classify(my_decision_tree, test_data.iloc[0], annotate=True)

Split on term_ 36 months = 0
Split on grade_A = 0
Split on grade_B = 0
Split on grade_C = 0
Split on grade_D = 1
At leaf, predicting -1


-1

**Quiz Question:** What was the feature that **my_decision_tree** first split on while making the prediction for test_data[0]?

**Answer**: term_ 36 months

**Quiz Question:** What was the first feature that lead to a right split of test_data[0]?

**Answer**: grade_D

**Quiz Question:** What was the last feature split on before reaching a leaf node for test_data[0]?

**Answer**: grade_D or grade_E ?

### Evaluating your decision tree

**16**. Now, we will write a function to evaluate a decision tree by computing the classification error of the tree on the given dataset.

Again, recall that the **classification error** is defined as follows:
$$
\mbox{classification error} = \frac{\mbox{# mistakes}}{\mbox{# total examples}}
$$

Now, write a function called `evaluate_classification_error` that takes in as input:
1. `tree` (as described above)
2. `data` (an SFrame)
3. `target` (a string - the name of the target/label column)

This function should calculate a prediction (class label) for each row in `data` using the decision `tree` and return the classification error computed using the above formula. Fill in the places where you find `## YOUR CODE HERE`. There is **one** place in this function for you to fill in.

In [51]:
def evaluate_classification_error(tree, data, target):
    # Apply the classify(tree, x) to each row in your data
    prediction = data.apply(lambda x: classify(tree, x), axis=1)
    
    # Once you've made the predictions, calculate the classification error and return it
    ## YOUR CODE HERE
    num_mistakes = sum(prediction != data[target])
    
    return num_mistakes / len(prediction)

Now, let's use this function to evaluate the classification error on the test set.

In [52]:
evaluate_classification_error(my_decision_tree, test_data, target)

0.3837785437311504

**Quiz Question:** Rounded to 2nd decimal point, what is the classification error of **my_decision_tree** on the **test_data**?

**Answer**: 0.38

### Printing out a decision stump

**18**. As discussed in the lecture, we can print out a single decision stump (printing out the entire tree is left as an exercise to the curious reader). 

In [55]:
def print_stump(tree, name = 'root'):
    split_name = tree['splitting_feature'] # split_name is something like 'term. 36 months'
    if split_name is None:
        print(f"(leaf, label: {tree['prediction']})")
        return None
    split_feature, split_value = split_name.split('.')
    print( '                       %s' % name)
    print('         |---------------|----------------|')
    print('         |                                |')
    print('         |                                |')
    print('         |                                |')
    print('  [{0} == 0]               [{0} == 1]    '.format(split_name))
    print('         |                                |')
    print('         |                                |')
    print('         |                                |')
    print('    (%s)                         (%s)' \
        % (('leaf, label: ' + str(tree['left']['prediction']) if tree['left']['is_leaf'] else 'subtree'),
           ('leaf, label: ' + str(tree['right']['prediction']) if tree['right']['is_leaf'] else 'subtree')))

**19**. Using this function, we can print out the root of our decision tree:

In [56]:
print_stump(my_decision_tree)

ValueError: not enough values to unpack (expected 2, got 1)